# Algorithms for Week 6

These algorithm will be used in the solutions for the week 6 Q&A questions.

## GP-UCB

In [ ]:
kernel0 = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))

def FitGP(myenv, kernel=kernel0):
    # define gp object that will be used to fit model
    #kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
    gp = GaussianProcessRegressor(kernel=kernel, alpha=myenv.var+1e-10, n_restarts_optimizer=1)
    
    # fit to data
    gp.fit(myenv.xhist.reshape(-1,1), myenv.yhist)
    
    return gp

#computes upper confidence bound at x
def UCB(gp, x, T, GPEnv):
    mean, sd = gp.predict(np.array(x).reshape(-1,1), return_std=True)
    beta = np.sqrt(2*np.log(np.pi**2*(GPEnv.xmax-GPEnv.xmin)*T*len(GPEnv.yhist)**2/6)) ##############################CHECK
    return mean + beta*sd

In [ ]:
def GPUCB(T,GPEnv,xmin,xmax,kernel=kernel0):
    myenv = GPEnv
    myenv.reset()
    
    # first sample must be random
    x = np.random.uniform(xmin,xmax)
    y = myenv.play(x)
    
    rew = np.zeros(T) 
    reg = np.zeros(T) 
    rew[0] = y
    reg[0] = myenv.maxrew - myenv.f(x)
    
    
    for t in range(1,T):
        gp = FitGP(myenv,kernel)
        #choose x with the largest UCB
        myx = fminbound(lambda x: -UCB(gp,x,T,myenv),xmin,xmax)
        y = myenv.play(myx)
        rew[t] = y
        reg[t] = myenv.maxrew - myenv.f(myx)

    return rew, reg

## GP Thompson Sampling

In [ ]:
def GP_Thompson(T,GPEnv,xmin,xmax,kernel=kernel0):
    myenv = GPEnv
    myenv.reset()
    # first sample must be random
    x = np.random.uniform(xmin,xmax)
    y = myenv.play(x)
    
    rew = np.zeros(T) 
    reg = np.zeros(T) 
    rew[0] = y
    reg[0] = myenv.maxrew - myenv.f(x)
    
    #define set over which we search for the best action
    search_set = np.linspace(myenv.xmin,myenv.xmax,1000)
    
    
    for t in range(1,T):
        gp = FitGP(myenv,kernel)
        #sample a function from the posterior, evaluate over search_set
        posterior_sample = gp.sample_y(search_set.reshape(-1, 1))[:,0]
        #play x with the largest function value
        myx_index = np.argmax(posterior_sample)
        myx = search_set[myx_index]
        y = myenv.play(myx)
        rew[t] = y
        reg[t] = myenv.maxrew - myenv.f(myx)

    return rew, reg

## Expected Improvement

In [ ]:
#computes expected improvement acquisition function at x
def EI(gp, x, T, GPEnv, fstar):
    #standard normal distribution
    rv = normal(0,1)
    #find mean and sd at x
    mean, sd = gp.predict(np.array(x).reshape(-1,1), return_std=True)
    #calculate acquisition function A at x
    A = max(mean-fstar,0) + sd*rv.pdf((mean-fstar)/sd) - np.absolute(mean-fstar)*rv.cdf((mean-fstar)/sd)
    return A

#runs expected improvement function with above acquisition function
def GP_ExpectedImprovement(T,GPEnv,xmin,xmax,kernel=kernel0):
    myenv = GPEnv
    myenv.reset()
    
    # first sample must be random
    x = np.random.uniform(xmin,xmax)
    y = myenv.play(x)
    
    rew = np.zeros(T) 
    reg = np.zeros(T) 
    rew[0] = y
    reg[0] = myenv.maxrew - myenv.f(x)
    
    
    for t in range(1,T):
        gp = FitGP(myenv,kernel)
        #choose x with the largest EI
        fstar = max(myenv.yhist)
        myx = fminbound(lambda x: -EI(gp,x,T,myenv,fstar),xmin,xmax)
        y = myenv.play(myx)
        rew[t] = y
        reg[t] = myenv.maxrew - myenv.f(myx)

    return rew, reg